In [5]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".XX"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
from jax.lib import xla_bridge
print("jax backend {}".format(xla_bridge.get_backend().platform))

from SSN_classes import SSN_mid_local
from training import train_model

jax backend cpu


In [6]:
import jax.numpy as np

from parameters import grid_pars, filter_pars, stimuli_pars, sig_pars, ssn_pars, conn_pars_m, conn_pars_s, ssn_layer_pars, conv_pars, training_pars, loss_pars

# check where A2 is defined in Clara's code
ssn_ori_map_loaded = np.load(os.path.join(os.getcwd(), 'ssn_map_uniform_good.npy'))
#Find normalization constant of Gabor filters
ssn_mid=SSN_mid_local(ssn_pars=ssn_pars, grid_pars=grid_pars, conn_pars=conn_pars_m, filter_pars=filter_pars, J_2x2= ssn_layer_pars.J_2x2_m, gE = ssn_layer_pars.gE_m, gI=ssn_layer_pars.gI_m, ori_map = ssn_ori_map_loaded)
ssn_pars.A = ssn_mid.A
ssn_pars.A2 = ssn_mid.A2
####################### TRAINING PARAMETERS #############################
constant_ssn_pars = dict(
    ssn_pars=ssn_pars,
    grid_pars=grid_pars,
    conn_pars_m=conn_pars_m,
    conn_pars_s=conn_pars_s,
    gE=ssn_layer_pars.gE,
    gI=ssn_layer_pars.gI,
    filter_pars=filter_pars,
    conv_pars=conv_pars,# this got a lot of new stuff in it that might be a problem
    loss_pars=loss_pars,
    noise_type="poisson",
)
#Collect constant parameters into single class
class constant_pars:
    ssn_pars = ssn_pars
    s_2x2 = ssn_layer_pars.s_2x2_s
    sigma_oris = ssn_layer_pars.sigma_oris
    grid_pars = grid_pars
    conn_pars_m = conn_pars_m
    conn_pars_s = conn_pars_s
    gE = ssn_layer_pars.gE
    gI = ssn_layer_pars.gI
    filter_pars = filter_pars
    noise_type = 'poisson'
    ssn_ori_map = ssn_ori_map_loaded
    ref_ori = stimuli_pars.ref_ori
    
readout_pars = dict(w_sig = sig_pars.w_sig, b_sig = sig_pars.b_sig)
ssn_layer_pars = dict(J_2x2_m = ssn_layer_pars.J_2x2_m, J_2x2_s = ssn_layer_pars.J_2x2_s, c_E = ssn_layer_pars.c_E, c_I = ssn_layer_pars.c_I, f_E = ssn_layer_pars.f_E, f_I = ssn_layer_pars.f_I, kappa_pre = ssn_layer_pars.kappa_pre, kappa_post = ssn_layer_pars.kappa_post)

########### TRAINING ############
from save_code import save_code
results_dir = save_code()
file_name = 'results.csv'
results_filename = os.path.join(results_dir,file_name)
(
        [ssn_layer_pars, readout_pars],
        val_loss_per_epoch,
        all_losses,
        train_accs,
        train_sig_input,
        train_sig_output,
        val_sig_input,
        val_sig_output,
        epochs_plot,
        save_w_sigs,
    ) = train_model(
    ssn_layer_pars,
    readout_pars,
    constant_pars,
    conv_pars,
    loss_pars,
    training_pars,
    stimuli_pars,
    results_filename
)

'''
([ssn_layer_pars_dict, readout_pars],
    val_loss_per_epoch,
    training_losses,
    training_accs,
    train_sig_inputs,
    train_sig_outputs,
    val_sig_inputs,
    val_sig_outputs,
    epoch_c,
    save_w_sigs) = new_two_stage_training(
    ssn_layer_pars,
    sig_pars,
    training_pars,
    constant_ssn_pars,
    stimuli_pars,
    extra_stop=2,
    ssn_ori_map=ssn_ori_map_loaded
    )
    '''

Files copied successfully to: results/Nov23_v0
Files copied successfully to: results/Nov23_v1
Training model for 5 epochs  with learning rate 0.001, sig_noise 2.0 at offset 4.0, lam_w 1, batch size 50, noise_type poisson
Saving results to csv  results/Nov23_v1\results.csv
Training loss: 1.338036060333252 ¦ Validation -- loss: 1.5556433200836182, true accuracy: 0.4399999976158142, at epoch 1, (time 50.96613907814026, 0.8963532447814941), 
Training loss: 1.6727712154388428 ¦ Validation -- loss: 1.5336360931396484, true accuracy: 0.5199999809265137, at epoch 3, (time 2.397770404815674, 1.1182911396026611), 
Training loss: 1.7069448232650757 ¦ Validation -- loss: 1.5380905866622925, true accuracy: 0.5, at epoch 5, (time 2.1164016723632812, 1.0537660121917725), 
Entering second stage at epoch 5
Training loss: 1.3961068391799927 ¦ Validation -- loss: 1.3785502910614014, true accuracy: 0.5199999809265137, at epoch 1, (time 2.1164016723632812, 2.701303720474243)
Training accuracy: 0.5400000214

'\n([ssn_layer_pars_dict, readout_pars],\n    val_loss_per_epoch,\n    training_losses,\n    training_accs,\n    train_sig_inputs,\n    train_sig_outputs,\n    val_sig_inputs,\n    val_sig_outputs,\n    epoch_c,\n    save_w_sigs) = new_two_stage_training(\n    ssn_layer_pars,\n    sig_pars,\n    training_pars,\n    constant_ssn_pars,\n    stimuli_pars,\n    extra_stop=2,\n    ssn_ori_map=ssn_ori_map_loaded\n    )\n    '

In [7]:
print(ssn_layer_pars['J_2x2_m'])

[[0.6551722  0.01111201]
 [1.2963108  0.5273492 ]]
